In [1]:
pwd

'/home/programmer/master_2/project/Decoupling-Representation-Learning-from-Reinforcement-Learning'

In [23]:
from typing import Sequence
import gym
import torch
import numpy as np
import copy
from stable_baselines3 import DQN


In [26]:
env = gym.make("LunarLander-v2")
policy_kwargs = {
         "net_arch": [64] * 2,
     }


policy_kwargs

{'net_arch': [64, 64]}

In [31]:
agent = DQN(env=env,policy_kwargs=policy_kwargs, policy = 'MlpPolicy', device='cpu')

In [45]:
feature_extractor = agent.q_net.features_extractor
latent_size = agent.q_net.features_dim

In [39]:
def extract_samples(
    seq_of_obs: torch.tensor,
    prediction_length_k: int, ):
    """
    each obs is allready a framesack, so we can just go util length - k
    """
    samples = []
    for i in range(seq_of_obs.shape[0] - prediction_length_k):
        anchor = seq_of_obs[i]
        positive = seq_of_obs[i + prediction_length_k]
        samples.append(torch.stack((anchor, positive), dim=0))
    return samples

In [56]:
env = gym.make("LunarLander-v2")
def create_dataset(env, episodes = 1, seed=0):
    samples = []
    env.seed(0)
    env.action_space.seed(0)
    for i_epiosde in range(episodes):
        obs = env.reset()
        seq_of_obs = torch.tensor(np.array(obs), dtype=torch.float32).unsqueeze(0)
        while True:
            action = env.action_space.sample()
            next_obs, reward, done, _ = env.step(action)
            if done:
                break
            seq_of_obs = torch.cat((seq_of_obs, torch.tensor(np.array(next_obs), dtype=torch.float32).unsqueeze(0),), dim=0)
    return extract_samples(seq_of_obs, 10)


In [105]:
device= "cpu"
dataset = create_dataset(env=env)
encoder = copy.deepcopy(feature_extractor).to(device)
target_encoder = copy.deepcopy(encoder).to(device)
forward_mlp = create_net(
            input_size=latent_size, output_size=latent_size
        ).to(device)
W = torch.nn.Linear(latent_size, latent_size, bias=False).to(device)
params = []
print("encoder params")
for param in encoder.parameters():
    print(type(param), param.size())
print("forward_mlp paramas")
for param in forward_mlp.parameters():
    print(type(param), param.size())
print("W params")
for param in W.parameters():
    print(type(param), param.size())

    
params += encoder.parameters()
params += forward_mlp.parameters()
params += W.parameters()
optim = torch.optim.Adam(params=params)
        
len(dataset)

encoder params
forward_mlp paramas
<class 'torch.nn.parameter.Parameter'> torch.Size([8, 8])
<class 'torch.nn.parameter.Parameter'> torch.Size([8])
W params
<class 'torch.nn.parameter.Parameter'> torch.Size([8, 8])


89

In [106]:
-


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [10]:
 def extract_samples(
        seq_of_obs: torch.tensor,
        prediction_length_k: int,
    ):
        """
        each obs is allready a framesack, so we can just go util length - k
        """
        samples = []
        for i in range(seq_of_obs.shape[0] - prediction_length_k):
            anchor = seq_of_obs[i]
            positive = seq_of_obs[i + prediction_length_k]
            samples.append(torch.stack((anchor, positive), dim=0))
        return samples

In [16]:
def create_net(input_size, output_size, hidden_sizes: Sequence = None):
        sequence = list()
        if hidden_sizes is None:
            hidden_sizes = []
        hidden_layers = [
            torch.nn.Linear(n_in, n_out)
            for n_in, n_out in zip([input_size] + hidden_sizes[:-1], hidden_sizes)
        ]

        for layer in hidden_layers:
            sequence.extend([layer, torch.nn.ReLU()])
        # add last layer without ReLU, also serves as first layer if we have no hidden layers
        last_size = input_size if hidden_sizes == [] else hidden_sizes[-1]
        sequence.append(torch.nn.Linear(last_size, output_size))
        return torch.nn.Sequential(*sequence)

(61,
 tensor([[-0.0025,  1.4028, -0.2551, -0.3607,  0.0029,  0.0578,  0.0000,  0.0000],
         [-0.0296,  1.3114, -0.2799, -0.4644,  0.0582,  0.1431,  0.0000,  0.0000]]))

In [22]:
device = "cpu"
latent_size = 8
encoder = 1
target_encoder = 0
forward_mlp = create_net(input_size=8, output_size=8).to(device)
W = torch.nn.Linear(latent_size, latent_size, bias=False).to(device)
forward_mlp

Sequential(
  (0): Linear(in_features=8, out_features=8, bias=True)
)

In [107]:
1e-3

0.001

In [9]:
seq_of_obs.shape

torch.Size([63, 8])

# class PreTrainer:
    def __init__(self, feature_extractor, latent_size, env, device=None) -> None:
        self.env = env
        self.dataset = self.create_dataset(env=self.env)
        self.encoder = copy.deepcopy(feature_extractor).to(device)
        self.target_encoder = copy.deepcopy(self.encoder).to(device)
        self.forward_mlp = self.create_net(
            input_size=latent_size, output_size=latent_size
        ).to(device)
        self.W = torch.nn.Linear(latent_size, latent_size, bias=False).to(device)
        params = []

        print("encoder params")
        for param in self.encoder.parameters():
            print(type(param), param.size())
        print("forward_mlp paramas")
        for param in self.forward_mlp.parameters():
            print(type(param), param.size())
        print("W params")
        for param in self.W.parameters():
            print(type(param), param.size())

        params += self.encoder.parameters()
        params += self.forward_mlp.parameters()
        params += self.W.parameters()
        self.optim = torch.optim.Adam(params=params)
        self.device = device
    
    def create_net(self, input_size, output_size, hidden_sizes: Sequence = None):
        sequence = list()
        if hidden_sizes is None:
            hidden_sizes = []
        hidden_layers = [
            torch.nn.Linear(n_in, n_out)
            for n_in, n_out in zip([input_size] + hidden_sizes[:-1], hidden_sizes)
        ]

        for layer in hidden_layers:
            sequence.extend([layer, torch.nn.ReLU()])
        # add last layer without ReLU, also serves as first layer if we have no hidden layers
        last_size = input_size if hidden_sizes == [] else hidden_sizes[-1]
        sequence.append(torch.nn.Linear(last_size, output_size))
        return torch.nn.Sequential(*sequence)
    
    def create_dataset(self, n_episodes: int = 1, env=None):
        samples = []
        for _ in range(n_episodes):
            obs = env.reset()
            # first to array to preserver fram stacking
            seq_of_obs = torch.tensor(np.array(obs), dtype=torch.float32).unsqueeze(0)
            while True:
                # we can sample from action space via env.action_space.sample(), effectively creating a random policy
                action = env.action_space.sample()
                next_obs, reward, done, info = env.step(action)
                if done:
                    break
                # we save it as flaot 32 as the deepset feature excractor is also float32
                seq_of_obs = torch.cat(
                    (
                        seq_of_obs,
                        torch.tensor(np.array(next_obs), dtype=torch.float32).unsqueeze(
                            0
                        ),
                    ),
                    dim=0,
                )
            # with this way of extracting samples we have a lot of redundancy,
            #  but as eposides have random length I see no clean way to store entore episodes and sample during training
            samples += self.extract_samples(
                seq_of_obs=seq_of_obs, prediction_length_k=10
            )  # should be in dependency of Ts
        return samples
    
    def extract_samples(
        self,
        seq_of_obs: torch.tensor,
        prediction_length_k: int,
    ):
        """
        each obs is allready a framesack, so we can just go util length - k
        """
        samples = []
        for i in range(seq_of_obs.shape[0] - prediction_length_k):
            anchor = seq_of_obs[i]
            positive = seq_of_obs[i + prediction_length_k]
            samples.append(torch.stack((anchor, positive), dim=0))
        return samples





In [3]:
env = gym.make("LunarLander-v2")
samples = []
n_episodes = 1
for _ in range(n_episodes):
    obs = env.reset()
    # first to array to preserver fram stacking

In [38]:
def contrast(anchor, positives):
        anchor = forward_mlp(anchor)
        pred = (anchor)
        logits = torch.matmul(pred, positives.T)
        # TODO we could normalize here
        # logits = logits - torch.max(logits, dim=1, keepdim=True)[0]  # normalize
        return logits


In [102]:
bs  = 512
grad_steps = 100000
info_freq  = 1000
tau = 0.9
device = "cpu"
latent_size = 8
encoder = 
target_encoder = 
forward_mlp = create_net(input_size=8, output_size=8).to(device)
W = torch.nn.Linear(latent_size, latent_size, bias=False).to(device)
forward_mlp
indices = np.arange(len(dataset))
choices = np.random.choice(indices, size=bs)
batch_elements = [dataset[choice] for choice in choices]
batch = torch.stack(batch_elements).to(device)
anchors = batch[:, 0]
positives = batch[:, 1]
c_anchors = encoder(anchors)
with torch.no_grad():
        c_positives = target_encoder(positives)
batch.shape, anchors.shape
logits = contrast(anchor=c_anchors, positives=c_positives)
labels = torch.arange(c_anchors.shape[0], dtype=torch.long, device=device)
# batch[0], anchors[0], positives[0]
ce_loss = torch.nn.CrossEntropyLoss()
loss = ce_loss(logits, labels)
c_anchors.shape , c_positives.shape, logits.shape,labels.shape, loss

TypeError: 'int' object is not callable

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn